# Simulate minimum deviation measurement of the refractive index

In [1]:
import numpy as np

## Initialize parameters

In [2]:
theta_deg = np.arange(45, 85, 5)
n0 = 1.67
alpha0_deg = 60
err_deg = 0.5 / 60

rng = np.random.default_rng(26)

## Define model function

In [3]:


def model(_theta_deg, n, alpha_deg):
    # Convert from degrees to radians, and treat as a complex variable
    theta = np.deg2rad(_theta_deg).astype(complex)
    alpha = np.deg2rad(alpha_deg)
    # Compute delta
    _delta = theta + np.arcsin(np.sin(alpha)
                               * np.sqrt(n**2 - np.sin(theta)**2)
                               - np.sin(theta) * np.cos(alpha)) - alpha
    # Warn the user if the argument of arcsin exceeds one
    if any(np.sin(alpha) * np.sqrt(n**2 - np.sin(theta)**2)
           - np.sin(theta) * np.cos(alpha)) > 1:
        msg = 'Argument to arcsin exceeds one, returning the complex norm'
        warnings.warn(msg)
    #  Take the absolute value and convert to back to degrees
    _delta_deg = np.rad2deg(np.absolute(_delta))
    return _delta_deg


## Simulate measurement

In [4]:
delta = (model(theta_deg, n0, alpha0_deg)
         + err_deg * rng.standard_normal(*theta_deg.shape))
delta_deg = np.floor(delta)
delta_min = np.round(60 * (delta-delta_deg))

data = np.column_stack((theta_deg, delta_deg, delta_min))

## Save data

In [5]:
np.savetxt('mindevdatasim.csv', data,
           fmt='%d',
           delimiter=',',
           header='theta (deg), delta (deg), delta (min)',
           comments='')